In [4]:
# ============================================================
# 📘 Document Image Preprocessing Pipeline v2 (F1-Optimized, Fixed)
# ============================================================
import os
import cv2
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

# -------------------- 1) 재현성 --------------------
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# -------------------- 2) 경로 --------------------
BASE_DIR = '/root/cv_project/data'
TRAIN_CSV = os.path.join(BASE_DIR, 'train.csv')
TEST_CSV  = os.path.join(BASE_DIR, 'sample_submission.csv')
TRAIN_IMG_DIR = os.path.join(BASE_DIR, 'train')
TEST_IMG_DIR  = os.path.join(BASE_DIR, 'test')
SAVE_DIR = os.path.join(BASE_DIR, 'processed_v2')
os.makedirs(SAVE_DIR, exist_ok=True)

# -------------------- 3) 유틸 --------------------
COMMON_EXTS = ['.jpg', '.jpeg', '.png', '.JPG', '.PNG', '.JPEG']

def has_ext(name: str) -> bool:
    return os.path.splitext(name)[1] != ''

def resolve_image_path(img_dir: str, name: str) -> str:
    """
    CSV의 id/file/path가
    - 절대경로
    - 상대경로
    - 확장자 포함/미포함
    - 확장자 대소문자 혼재
    전부를 안전하게 처리하여 실제 존재하는 경로를 반환.
    """
    # 절대경로로 이미 존재?
    if os.path.isabs(name) and os.path.exists(name):
        return name

    base_name = os.path.basename(str(name))
    # 1) 그대로 결합
    cand = os.path.join(img_dir, base_name)
    candidates = [cand]

    root, ext = os.path.splitext(base_name)
    if has_ext(base_name):
        # 확장자 바꿔가며 존재 확인 (csv 확장자와 실제 파일 확장자가 다를 수 있음)
        for e in COMMON_EXTS:
            candidates.append(os.path.join(img_dir, root + e))
    else:
        # 확장자 없으면 대표 확장자 붙여가며 확인
        for e in COMMON_EXTS:
            candidates.append(os.path.join(img_dir, base_name + e))

    # 중복 제거하며 존재 확인
    seen = set()
    for p in candidates:
        if p in seen: 
            continue
        seen.add(p)
        if os.path.exists(p):
            return p

    # 디버깅 도움: 시도한 후보 안내
    raise FileNotFoundError(
        f"이미지 파일을 찾지 못했습니다. name='{name}', img_dir='{img_dir}'\n"
        f"시도한 후보 예: {candidates[:5]} ..."
    )

# -------------------- 4) 전처리 함수 (lambda 대체) --------------------
def deskew_cv(image, **kwargs):   # ← **kwargs 추가
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coords = np.column_stack(np.where(gray > 0))
    if coords.size == 0:
        return image
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),
                             flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def apply_clahe_cv(image, **kwargs):   # ← **kwargs 추가
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

def resize_with_padding_512(image, **kwargs):   # ← **kwargs 추가
    target_size = 512
    h, w = image.shape[:2]
    if max(h, w) == 0:
        return cv2.resize(image, (target_size, target_size))
    scale = target_size / max(h, w)
    nh, nw = int(h * scale), int(w * scale)
    resized = cv2.resize(image, (nw, nh))
    pad_h = (target_size - nh) // 2
    pad_w = (target_size - nw) // 2
    padded = cv2.copyMakeBorder(resized,
                                pad_h, target_size - nh - pad_h,
                                pad_w, target_size - nw - pad_w,
                                cv2.BORDER_CONSTANT, value=255)
    return padded



# -------------------- 5) Albumentations (lambda 제거) --------------------
train_transform = A.Compose([
    A.Lambda(image=deskew_cv, p=0.5),
    A.Lambda(image=apply_clahe_cv, p=0.5),
    A.RandomShadow(p=0.3),
    A.RandomBrightnessContrast(p=0.4),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.3),
    A.GaussNoise(p=0.3),
    A.Perspective(scale=(0.02, 0.05), p=0.3),
    A.Rotate(limit=8, border_mode=cv2.BORDER_REPLICATE, p=0.5),
    A.Lambda(image=resize_with_padding_512, p=1.0),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Lambda(image=deskew_cv, p=1.0),
    A.Lambda(image=apply_clahe_cv, p=1.0),
    A.Lambda(image=resize_with_padding_512, p=1.0),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

# -------------------- 6) Dataset --------------------
class DocumentDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.id_col = [c for c in self.df.columns if any(k in c.lower() for k in ['id','file','path','image'])]
        self.label_col = [c for c in self.df.columns if c.lower() in ['label','target','class']]
        self.id_col = self.id_col[0] if self.id_col else self.df.columns[0]
        self.label_col = self.label_col[0] if self.label_col else None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        name = str(row[self.id_col])
        img_path = resolve_image_path(self.img_dir, name)   # ✅ 안전 경로 탐색 (확장자 이중 방지)
        img = np.array(Image.open(img_path).convert("RGB"))
        label = int(row[self.label_col]) if self.label_col else -1
        if self.transform:
            img = self.transform(image=img)['image']
        return img, label

# -------------------- 7) 저장 --------------------
def save_preprocessed_tensors(dataset, name="train", batch_size=32, num_workers=0):
    # ⚠ lambda 경고 회피: num_workers=0 권장(또는 위에서 lambda 제거했으니 2~4도 가능)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    all_images, all_labels = [], []

    print(f"🔹 {name} 데이터 전처리 중...")
    for imgs, labels in tqdm(loader):
        all_images.append(imgs)
        all_labels.append(labels)

    all_images = torch.cat(all_images)
    all_labels = torch.cat(all_labels)
    torch.save({'images': all_images, 'labels': all_labels},
               os.path.join(SAVE_DIR, f"{name}_processed.pt"))
    print(f"✅ {name}_processed.pt 저장 완료 ({all_images.shape[0]}개 이미지)")

# -------------------- 8) 실행 --------------------
if __name__ == "__main__":
    print("📘 고도화된 문서 이미지 전처리 파이프라인 시작")

    train_ds = DocumentDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=train_transform)
    test_ds  = DocumentDataset(TEST_CSV,  TEST_IMG_DIR,  transform=test_transform)

    # 첫 실행은 num_workers=0 권장 (경고/경로 문제 확실히 해결 후 늘려도 됨)
    save_preprocessed_tensors(train_ds, name="train", num_workers=0)
    save_preprocessed_tensors(test_ds,  name="test",  num_workers=0)

    print("✨ 모든 전처리 완료 및 저장 완료 (F1-Optimized v2, fixed)")


📘 고도화된 문서 이미지 전처리 파이프라인 시작
🔹 train 데이터 전처리 중...


100%|██████████| 50/50 [00:40<00:00,  1.24it/s]


✅ train_processed.pt 저장 완료 (1570개 이미지)
🔹 test 데이터 전처리 중...


100%|██████████| 99/99 [01:11<00:00,  1.38it/s]


✅ test_processed.pt 저장 완료 (3140개 이미지)
✨ 모든 전처리 완료 및 저장 완료 (F1-Optimized v2, fixed)
